<div class="alert alert-block alert-success">
<b> Comentário Geral do revisor: </b> <a class="tocSkip"></a>

Tudo certo agora. Porém, destaco  a importância de fazer EDA antes de fazer imputação nos dados. Não sei se você conhece a técnica SMOTE; vale a pena ler sobre na internet pois é comum e utilizada para desbalanceamento. Desejo sucesso na jornada. Parabéns pelo empenho.
</div>


## Realizando importações

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score,roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


## Verificando os dados

In [2]:
data =pd.read_csv("datasets/Churn.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [3]:
print(data.duplicated().sum())

0


In [4]:
print(data.head())

   RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age  \
0          1    15634602  Hargrave          619    France  Female   42   
1          2    15647311      Hill          608     Spain  Female   41   
2          3    15619304      Onio          502    France  Female   42   
3          4    15701354      Boni          699    France  Female   39   
4          5    15737888  Mitchell          850     Spain  Female   43   

   Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMember  \
0     2.0       0.00              1          1               1   
1     1.0   83807.86              1          0               1   
2     8.0  159660.80              3          1               0   
3     1.0       0.00              2          0               0   
4     2.0  125510.82              1          1               1   

   EstimatedSalary  Exited  
0        101348.88       1  
1        112542.58       0  
2        113931.57       1  
3         93826.63       0  
4         790

## tratando dados

### Adequando dados e separando caracteristicas e alvos

In [5]:
data['Tenure'] = data['Tenure'].fillna(0)

para manter a maior quantidade possivel de dados os valores que estavam ausentes foram subistituidos 0, supondo que os valores auzentes não tenham debito fixo

In [6]:
data = pd.get_dummies(data.drop(['Surname'],axis=1),drop_first=True)

In [7]:
target=data['Exited']
features= data.drop(['RowNumber','Exited'],axis=1)

### Verificando desequilibrio

In [8]:
print(target.value_counts(normalize=True))

Exited
0    0.7963
1    0.2037
Name: proportion, dtype: float64


### Separando grupos de treino, teste e validação

In [9]:
features_train, features_valid,target_train,target_valid = train_test_split(features,target,test_size=0.4,random_state=12345)
features_valid, features_test,target_valid,target_test = train_test_split(features,target,test_size=0.50,random_state=12345)

### Testando diferentes modelos

In [10]:
model = LogisticRegression(random_state=12345,solver='liblinear',class_weight='balanced')
model.fit(features_train,target_train)
predicted_test = model.predict(features_test)
print(f1_score(target_test,predicted_test))

0.4869689396644056


In [11]:
model = DecisionTreeClassifier(random_state=12345,max_depth=10)
model.fit(features_train,target_train)
predicted_test = model.predict(features_test)
print(f1_score(target_test,predicted_test))

0.5585480093676815


In [12]:
model = RandomForestClassifier(random_state=12345,max_depth=5,n_estimators=100)
model.fit(features_train,target_train)
predicted_test = model.predict(features_test)
print(f1_score(target_test,predicted_test))

0.4414090582314881


Dos modelos de testados podemos notar que o modelo com maior eficacia foi o modelo de arvore, apresentando o maior f1_score, indicando um maior equilibrio entre precisão e recall

## Corrigindo desequilibrio

In [13]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)]
        + [features_ones]
    )
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)]
        + [target_ones]
    )

    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345
    )

    return features_downsampled, target_downsampled


features_downsampled, target_downsampled = downsample(
    features_train, target_train, 0.1
)

model.fit(features_downsampled, target_downsampled)
predicted_valid = model.predict(features_valid)
print(f1_score(target_valid,predicted_valid))
print()
print(target_downsampled.value_counts())

0.4364763169544046

Exited
1    1196
0     480
Name: count, dtype: int64


In [14]:
def upsample(features,target,repeat):
    features_zeros = features[target==0]
    features_ones = features[target==1]
    target_zeros = target[target==0]
    target_ones = target[target==1]
    
    features_upsampled = pd.concat([features_zeros]+[features_ones]*repeat)
    target_upsampled = pd.concat([target_zeros]+[target_ones]*repeat)
    
    features_upsampled,target_upsampled = shuffle(features_upsampled,target_upsampled, random_state=12345)
    
    return features_upsampled,target_upsampled


features_upsampled, target_upsampled = upsample(features_train, target_train, 10)

model = DecisionTreeClassifier(random_state=12345,max_depth=10)
model.fit(features_upsampled, target_upsampled)
predicted_valid = model.predict(features_valid)
print(f1_score(target_valid,predicted_valid))
print()
print(target_upsampled.value_counts())

0.624720536569786

Exited
1    11960
0     4804
Name: count, dtype: int64


Foi feito o uso de superamostragem e subamostragem, onde a superamostragem se mostrou mais eficiente, apresentando um f1_score de 0.62

In [15]:
probabilit_valid = model.predict_proba(features_valid)
probabilities_valid_one = probabilit_valid[:,1]
print(roc_auc_score(target_valid,probabilities_valid_one))

0.9417372986038643


Podemos notar que o valor roc-auc esta proximo de 1 (0.94), o que indica uma boa performasse ao distinguir classes. Ou seja raramente vai avaliar uma classe negativa como positiva e o mesmo vale para o contrario

## Conclusão

Validando os modelos e aplicando algumas tecnicas para corrigir o desbalanceamento, notamos que o melhor desempenho veio com a arvore de decisão utilizando uma superamostragem. resultando em um f1 de 0.62, acima do minimo esperado, e um auc-roc de 0.94 um valor proximo de um indicando um bom desempenho